In [48]:
import web3
from web3 import Web3

# Define the ABI
abi = [
    {
        "inputs": [],
        "stateMutability": "nonpayable",
        "type": "constructor"
    },
    {
        "anonymous": False,
        "inputs": [
            {
                "indexed": False,
                "internalType": "string",
                "name": "key",
                "type": "string"
            },
            {
                "indexed": False,
                "internalType": "uint128",
                "name": "value",
                "type": "uint128"
            },
            {
                "indexed": False,
                "internalType": "uint128",
                "name": "timestamp",
                "type": "uint128"
            }
        ],
        "name": "OracleUpdate",
        "type": "event"
    },
    {
        "anonymous": False,
        "inputs": [
            {
                "indexed": False,
                "internalType": "address",
                "name": "newUpdater",
                "type": "address"
            }
        ],
        "name": "UpdaterAddressChange",
        "type": "event"
    },
    {
        "inputs": [
            {
                "internalType": "string",
                "name": "key",
                "type": "string"
            }
        ],
        "name": "getValue",
        "outputs": [
            {
                "internalType": "uint128",
                "name": "",
                "type": "uint128"
            },
            {
                "internalType": "uint128",
                "name": "",
                "type": "uint128"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "string[]",
                "name": "keys",
                "type": "string[]"
            },
            {
                "internalType": "uint256[]",
                "name": "compressedValues",
                "type": "uint256[]"
            }
        ],
        "name": "setMultipleValues",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "string",
                "name": "key",
                "type": "string"
            },
            {
                "internalType": "uint128",
                "name": "value",
                "type": "uint128"
            },
            {
                "internalType": "uint128",
                "name": "timestamp",
                "type": "uint128"
            }
        ],
        "name": "setValue",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "address",
                "name": "newOracleUpdaterAddress",
                "type": "address"
            }
        ],
        "name": "updateOracleUpdaterAddress",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [
            {
                "internalType": "string",
                "name": "",
                "type": "string"
            }
        ],
        "name": "values",
        "outputs": [
            {
                "internalType": "uint256",
                "name": "",
                "type": "uint256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    }
]

# Define the log data
log_data = {
    "data": "0x0000000000000000000000000000000000000000000000000000048c27395000",
    "topics": [
      "0x3115d1449a7b732c986cba18244e897a450f61e1bb8d589cd2e69e6c8924f9f7",
      "0x0000000000000000000000000000000000000000000000000000000100000005",
      "0x0000000000000000000000001232508adcaf57c6e78a850f9d715e3694b52000",
      "0x0000000000000000000000001232508adcaf57c6e78a850f9d715e3694b52000"
    ],
    "address": "0x1b02e051683b5cfac5929c25e84adb26ecf87b38"
  }





In [49]:
def decode_log_with_abi(log_data, contract_abi):
    w3 = Web3()

    # Extract event signature (first topic)
    event_signature_hex = log_data["topics"][0]

    # Find matching event in ABI
    matching_event = None
    for item in contract_abi:
        if item["type"] == "event":
            # Generate event signature hash
            args_types = [inp["type"] for inp in item["inputs"]]
            event_signature = f"{item['name']}({','.join(args_types)})"
            computed_hash = w3.keccak(text=event_signature).hex()

            if computed_hash == event_signature_hex:
                matching_event = item
                break

    if not matching_event:
        available_events = [e["name"] for e in contract_abi if e.get("type") == "event"]
        print(f"No matching event found. Available events: {available_events}")
        matching_event = "OracleUpdate"
        # raise ValueError(f"No matching event found. Available events: {available_events}")

    # Create dummy log entry with required fields
    log_entry = {
        "topics": [bytes.fromhex(t[2:]) for t in log_data["topics"]],
        "data": bytes.fromhex(log_data["data"][2:]),
        "logIndex": 0,
        "transactionIndex": 0,
        "transactionHash": b'\x00' * 32,
        "address": log_data["address"],
        "blockHash": b'\x00' * 32,
        "blockNumber": 0
    }

    # Decode using the matched event
    event_abi = next(
        item for item in abi
        if item.get("type") == "event"
        and item.get("name") == "OracleUpdate"
    )
    print(event_abi)
    contract = w3.eth.contract(abi=[event_abi])
    contract_event = contract.events.OracleUpdate()

    # event_class = getattr(contract.events, matching_event["name"])
    decoded_log = contract_event().process_log(log_entry)
    print(decoded_log)
    # return {
    #     "event_name": matching_event["name"],
    #     "args": dict(decoded.args),
    #     "raw_log": decoded
    # }

In [50]:


# Usage
decoded = decode_log_with_abi(log_data, abi)
print(f"Decoded {decoded['event_name']}:")
for arg, value in decoded["args"].items():
    print(f"{arg}: {value}")

No matching event found. Available events: ['OracleUpdate', 'UpdaterAddressChange']
{'anonymous': False, 'inputs': [{'indexed': False, 'internalType': 'string', 'name': 'key', 'type': 'string'}, {'indexed': False, 'internalType': 'uint128', 'name': 'value', 'type': 'uint128'}, {'indexed': False, 'internalType': 'uint128', 'name': 'timestamp', 'type': 'uint128'}], 'name': 'OracleUpdate', 'type': 'event'}
{'args': {'key': 'WBTC/USD', 'value': 5708660594118, 'timestamp': 1720771575}, 'event': 'OracleUpdate', 'logIndex': 0, 'transactionIndex': 0, 'transactionHash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'address': '0xdee629af973ebf5bf261ace12ffd1900ac715f5e', 'blockHash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'blockNumber': 0}


TypeError: 'NoneType' object is not subscriptable